# Add the capacity to verify the board

Checking the board visually is fine for debugging but it does not work well if we want to test a lot of board solutions

Goals:
- given a board is it completed ? 
- Are cells left undefined ? 
- Does it holds the clues ?

## Board class from notebook 01

In [1]:
import numpy as np
from enum import Enum

# constants for mark lette
class BoardMark(Enum):
    INIT = -1
    FILLER = 0
    BLACK = 1

class Board:
    '''
    Manage the board.
    '''
    
    def __init__(self, some_clues):
        '''
        board constructor
        '''
        # given parameters
        self.clues = some_clues
        # compute board dimensions
        self.width = len(self.clues['cols']) 
        self.height = len(self.clues['rows']) 
        # create board of type int, initialized with -1
        default_value = BoardMark.INIT.value
        self.states = np.full((self.height, self.width), default_value, dtype=int)
        
    def mark(self, row, col, mark):
        '''
        mark a cell
        '''
        self.states[row, col] = mark.value

    def prettyprint(self):
        '''
        pretty print of the board 
        '''
        print("cols:", end=" ")
        print(*self.clues['cols']) # * unpacks and remove []s
        print("rows:")
        [print(r) for r in self.clues['rows']]
          
        marks_switcher = {-1: '.', 0: 'x', 1: 'o'}
        # for whatever reason lambda does not work here
        def f_marks(v):
            return marks_switcher[v]
        applyall = np.vectorize(f_marks)
        marks = applyall(self.states)
        print(marks)


In [2]:
# clues for mini game
clues = {
    'rows': [1, 2],
    'cols': [2, 1]
}
board = Board(clues)
board.prettyprint()
board.mark(0, 1, BoardMark.FILLER) 
board.mark(1, 1, BoardMark.BLACK) 
board.prettyprint()

cols: 2 1
rows:
1
2
[['.' '.']
 ['.' '.']]
cols: 2 1
rows:
1
2
[['.' 'x']
 ['.' 'o']]


## check numbers of cells left

In [3]:
[ c for c in board.states.reshape(4) if c == -1]

[-1, -1]

In [7]:
count_left = len([ c for c in board.states.reshape(4) if c == -1])
print(f"count_left={count_left}")
assert count_left == 2

count_left=2


## check whether board hold the clues

In [45]:
# clues for mini game
clues = {
    'rows': [1, 2],
    'cols': [2, 1]
}
board = Board(clues)
board.prettyprint()
board.mark(0, 0, BoardMark.BLACK) 
board.mark(0, 1, BoardMark.FILLER) 
board.mark(1, 0, BoardMark.BLACK) 
board.mark(1, 1, BoardMark.BLACK) 
board.prettyprint()

cols: 2 1
rows:
1
2
[['.' '.']
 ['.' '.']]
cols: 2 1
rows:
1
2
[['o' 'x']
 ['o' 'o']]


In [46]:
lines = board.states
lines

array([[1, 0],
       [1, 1]])

In [47]:
from itertools import groupby

In [48]:
# group cells by value and select groups where group key is 1 (BLACK)
[ [(k, *g) for k,g in groupby(line) if k==1]
  for line in lines
]

[[(1, 1)], [(1, 1, 1)]]

In [66]:
blocks = [ [len(list(g)) for k,g in groupby(line) if k==1]
  for line in lines
]
print(f"blocks={blocks}")
assert blocks == [[1], [2]]

blocks=[[1], [2]]


In [53]:
# we actually need to compare with [1, 2]
[[clue] for clue in clues['rows'] ]

[[1], [2]]

In [59]:
[[clue] for clue in [1, 3, [1,1], 3, 5]]

[[1], [3], [[1, 1]], [3], [5]]

In [61]:
f = lambda clue: clue if isinstance(clue, list) else [clue]
[f(clue) for clue in [1, 3, [1,1], 3, 5]]

[[1], [3], [1, 1], [3], [5]]

In [63]:
f = lambda clue: clue if isinstance(clue, list) else [clue]

In [65]:
[f(clue) for clue in clues['rows'] ]

[[1], [2]]

In [67]:
assert blocks == [f(clue) for clue in clues['rows'] ]

In [68]:
w = 2
h = 2
cols = [ [board.states.reshape(w*h)[c+r*w]
         for r in range(0, h)]
         for c in range(0, w)]
cols

[[1, 1], [0, 1]]

In [69]:
blocks = [ [len(list(g)) for k,g in groupby(line) if k==1]
  for line in cols
]
print(f"blocks={blocks}")
assert blocks == [[2], [1]]

blocks=[[2], [1]]


In [71]:
assert blocks == [f(clue) for clue in clues['cols'] ]

## updated class

In [108]:
import numpy as np
from enum import Enum
from itertools import groupby

# constants for mark lette
class BoardMark(Enum):
    INIT = -1
    FILLER = 0
    BLACK = 1

class Board:
    '''
    Manage the board.
    '''
    
    def __init__(self, some_clues):
        '''
        board constructor
        '''
        # given parameters
        self.clues = some_clues
        # compute board dimensions
        self.width = len(self.clues['cols']) 
        self.height = len(self.clues['rows']) 
        self.count_cells = self.width * self.height
        # create board of type int, initialized with -1
        default_value = BoardMark.INIT.value
        self.states = np.full((self.height, self.width), default_value, dtype=int)
        
    def mark(self, row, col, mark):
        '''
        mark a cell
        '''
        self.states[row, col] = mark.value

    def prettyprint(self):
        '''
        pretty print of the board 
        '''
        print("cols:", end=" ")
        print(*self.clues['cols']) # * unpacks and remove []s
        print("rows:")
        [print(r) for r in self.clues['rows']]
          
        marks_switcher = {-1: '.', 0: 'x', 1: 'o'}
        # for whatever reason lambda does not work here
        def f_marks(v):
            return marks_switcher[v]
        applyall = np.vectorize(f_marks)
        marks = applyall(self.states)
        print(marks)

    def count_cells_left(self):
        return len([ c 
                  for c in board.states.reshape(self.count_cells) 
                  if c == BoardMark.INIT.value])


    def does_hold_clues(self):

        f = lambda clue: clue if isinstance(clue, list) else [clue] 
        
        rows = self.states
        rows_blocks = [ [len(list(g)) for k,g in groupby(row) if k==1]
          for row in rows
        ]
        rows_status = rows_blocks == [f(clue) for clue in clues['rows'] ]
        
        cols = [ [self.states.reshape(self.count_cells)[c+r*self.width]
         for r in range(0, self.height)]
         for c in range(0, self.width)]
        cols_blocks = [ [len(list(g)) for k,g in groupby(col) if k==1]
          for col in cols
        ]
        cols_status = cols_blocks == [f(clue) for clue in clues['cols'] ]

        return rows_status and cols_status
    
    
    def is_done(self):
        no_cells_left = self.count_cells_left() == 0
        return no_cells_left and self.does_hold_clues()


In [109]:
# clues for mini game
clues = {
    'rows': [1, 2],
    'cols': [2, 1]
}
board = Board(clues)
board.prettyprint()
board.mark(0, 0, BoardMark.BLACK) 
board.mark(0, 1, BoardMark.FILLER) 
board.mark(1, 0, BoardMark.BLACK) 
board.mark(1, 1, BoardMark.BLACK) 
board.prettyprint()

cols: 2 1
rows:
1
2
[['.' '.']
 ['.' '.']]
cols: 2 1
rows:
1
2
[['o' 'x']
 ['o' 'o']]


In [86]:
assert board.count_cells_left() == 0

In [84]:
assert board.does_hold_clues() 

In [110]:
assert board.is_done() 

## check 5x5 board

In [111]:
clues = {
    'rows': [1, 3, [1,1], 3, 5],
    'cols': [1, 4, [2,2], 4, 1]
}
board = Board(clues)

# row 4 is ooooo - only option for 5
board.states[4, :] = BoardMark.BLACK.value
# col 2 is ooxoo - only option for [2, 2]
board.states[:, 2] = BoardMark.BLACK.value
board.states[2, 2] = BoardMark.FILLER.value
# col 1 and 3 are xoooo - continguous to the placed black
board.states[:, 1] = BoardMark.BLACK.value
board.states[0, 1] = BoardMark.FILLER.value
board.states[:, 3] = BoardMark.BLACK.value
board.states[0, 3] = BoardMark.FILLER.value
# rows 1 and 3 are xooox - 3 blacks are placed
board.states[1, 0] = BoardMark.FILLER.value
board.states[1, 4] = BoardMark.FILLER.value
board.states[3, 0] = BoardMark.FILLER.value
board.states[3, 4] = BoardMark.FILLER.value
# row 2 is xoxox - [1, 1] blacks are placed
board.states[2, 0] = BoardMark.FILLER.value
board.states[2, 4] = BoardMark.FILLER.value
# row 0 is xoxox - 1 black is placed
board.states[0, 0] = BoardMark.FILLER.value
board.states[0, 4] = BoardMark.FILLER.value

board.prettyprint()


cols: 1 4 [2, 2] 4 1
rows:
1
3
[1, 1]
3
5
[['x' 'x' 'o' 'x' 'x']
 ['x' 'o' 'o' 'o' 'x']
 ['x' 'o' 'x' 'o' 'x']
 ['x' 'o' 'o' 'o' 'x']
 ['o' 'o' 'o' 'o' 'o']]


In [112]:
assert board.is_done() 